In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import portfolio_management_tools as pmt
from dotenv import load_dotenv

import yfinance as yf
from pandas_datareader import data as pdr


import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm
import scipy.stats as stats

In [2]:
yf.pdr_override()

In [58]:
def market_cap_df(stock_list, start, end):
    price_df = pd.DataFrame()
    cap_df = pd.DataFrame()
    data = yf.Tickers(stock_list)
    for i in stock_list:
        price = data.tickers[i].history(start = start, end = end)[['Close']]
        price_df = pd.concat([price_df, price], axis = 1)
        
        shares_df = pd.DataFrame(data.tickers[i].get_shares_full(start = start, end = end), columns = [i]).reset_index()
        shares_df.columns = ['Date', 'Shares']
        shares_df.set_index('Date', inplace = True)
        shares_df = shares_df[~shares_df.index.duplicated(keep = 'first')]
        
        temp_df = pd.merge(price, shares_df, on = 'Date', how = 'left')
        temp_df.fillna(method = 'ffill', inplace = True)
        temp_df.fillna(method = 'bfill', inplace = True)
        temp_df[i] = temp_df['Close'] * temp_df['Shares']
        
        cap_df = pd.concat([cap_df, temp_df[i]], axis = 1)
    price_df.columns = stock_list
    price_df.interpolate(method='linear', limit_direction='forward', axis=0)
    
    cap_last_day = cap_df.iloc[-1:,:]
        
    rf_df = pd.read_csv('Resources_from_others/10_year_tresury_1980_to_2024.csv')
    rf_df.columns = ['Date', 'Risk free rate (daily)']
    rf_df['Date'] = pd.to_datetime(rf_df['Date']).dt.date
    rf_df.set_index('Date', inplace = True)
    rf_df[rf_df['Risk free rate (daily)'] == '.'] = np.nan
    rf_df['Risk free rate (daily)'] = rf_df['Risk free rate (daily)'].astype('float64') / (252 * 100)
    
    tickers = yf.Tickers(['^GSPC'])
    market_return_df = tickers.tickers['^GSPC'].history(start = start, end = end)[['Close']]
    market_return_df = market_return_df.pct_change().dropna()
    market_return_df.reset_index(inplace = True)
    market_return_df.Date = market_return_df.Date.dt.date
    market_return_df.set_index('Date', inplace = True)

    return price_df, cap_last_day, rf_df, market_return_df

def market_weight(cap_last_day):
    w_mkt = cap_last_day / cap_last_day.sum(axis = 1).values[0]
    w_mkt['Market Cap'] = 'Market_cap'
    w_mkt.set_index('Market Cap', inplace = True)
    return w_mkt.T

def cov_matrix(price_df):
    return price_df.pct_change().cov()

def lambda_(market_return_df, risk_free_df):
    merged_df = pd.merge(market_return_df, risk_free_df, on = 'Date', how = 'left')
    merged_df['Excess market return'] = merged_df['Close'] - merged_df['Risk free rate (daily)']
    lambd = merged_df['Excess market return'].mean() / (merged_df['Close'].var())
    return lambd


def implied_return(price_df, cap_last_day, rf_df, market_return_df):
    w_mkt = market_weight(cap_last_day)
    sigma = cov_matrix(price_df)
    lmbda = lambda_(market_return_df, rf_df)
    index = w_mkt.index
    implied_return_dataframe = pd.DataFrame(lmbda * np.dot(sigma, w_mkt) * 252, index = index, columns = ['Annual implied returns'])
    return implied_return_dataframe



def create_row(stock_list):
    final_list = np.zeros(len(stock_list))
    
    valid_total_weights = [-100, 0, 100]
    
    while True:
        affected_stocks = input("Please write the number of stocks that are affected: ")
        try:
            affected_stocks = int(affected_stocks)
            if affected_stocks <= 0 or affected_stocks > len(stock_list):
                raise ValueError("Number of affected stocks must be within the range of available stocks.")
        except ValueError as e:
            print(e)
        else:
            break
    print('***********')    
    
    while True:
        positive_weights = 0
        negative_weights = 0
        lst = []
    
        for i in range(affected_stocks):
            while True:  # Loop for stock symbol input
                stock_symbol = input('Please write the affected stock symbol: ')
                if stock_symbol not in stock_list:
                    print('Invalid stock symbol. Please try again.')
                elif stock_symbol in lst:
                    print('You have entered this stock already. Please try again.')
                else:
                    break  # Valid stock symbol, exit this loop
            lst.append(stock_symbol)
            while True:  # Loop for weight input
                try:
                    weight = input('Please enter weight in percentage: ')
                    weight = int(weight)
                    if weight > 100 or weight < -100 or weight == 0:
                        raise ValueError('Invalid weight. Weight must be between -100 and 100, excluding 0.')
                except ValueError as e:
                    print(e)
                else:
                    break
            if weight < 0:
                negative_weights += weight
                index = stock_list.index(stock_symbol)
                final_list[index] = weight
            elif weight > 0:
                positive_weights += weight
                index = stock_list.index(stock_symbol)
                final_list[index] = weight
            print('----------')
        if positive_weights in valid_total_weights and negative_weights in valid_total_weights:
            break
        else:
            print('The sum of positive and negative weights is not valid. Please start over.')
    
    final_list = final_list / 100
    
    return final_list


def view_matrix_and_vector(stock_list):
    final_matrix = np.array([])
    view_vector = np.array([])
    while True:
        num_of_views = input('How many view are there?: ')
        try:
            num_of_views = int(num_of_views)
            if num_of_views <= 0:
                raise ValueError('Number you have entered is invalid.')
        except ValueError as e:
            print(e)
        else:
            break
    print('********')
    for i in range(num_of_views):
        print(f'View {i + 1}')
        
        while True:
            view_element = input('By how much will the view affect the stocks in percentage?')
            try:
                view_element = float(view_element)
                if i == 0:
                    view_vector = np.array([view_element / 100])
                else:
                    view_vector = np.vstack([view_vector, view_element / 100])
                
            except ValueError as e:
                print(e)
            else:
                break
        
        row = create_row(stock_list)
        if i == 0:
            final_matrix = row
        else:
            final_matrix = np.vstack([final_matrix, row])
    
    final_dataframe = pd.DataFrame(final_matrix, columns = stock_list)
    return final_dataframe, view_vector

def omega(tau, P, sigma):
    return tau * np.dot(np.dot(P, sigma), P.T)

In [66]:
def black_litterman_returns(stock_list, start, end, tau = 1, Z_score = True):
    price_df, cap_last_day, rf_df, market_return_df = market_cap_df(stock_list, start, end)
    P, Q = view_matrix_and_vector(stock_list)
    sigma = cov_matrix(price_df)
    returns = implied_return(price_df, cap_last_day, rf_df, market_return_df)
    Om = omega(tau, P, sigma)
    
    first_term = np.linalg.inv(np.linalg.inv(tau * sigma) + np.dot(np.dot(P.T, np.linalg.inv(Om)), P))
    second_term = np.dot(np.linalg.inv(tau * sigma), returns) + np.dot(np.dot(P.T, np.linalg.inv(Om)), Q)
    
    bl_returns = np.dot(first_term, second_term)
    bl_returns = pd.DataFrame(bl_returns, index = stock_list, columns = ['Black_litterman return'])
    
    if Z_score == True:
        Z = np.dot(np.linalg.inv(sigma * 252), bl_returns)
        Z = pd.DataFrame(Z, index = stock_list, columns = ['Z score'])
        
    
    return bl_returns, Z


In [4]:
# Making S&P 500 list to grab data
snp_df = pd.read_csv('Resources_from_others/S&P_500_constituents.csv')
snp_df.Symbol[snp_df.Symbol == 'BRK.B'] = 'BRK-B'
snp_df.Symbol[snp_df.Symbol == 'BF.B'] = 'BF-B'
snp_tickers = snp_df.Symbol.tolist()
snp_tickers.remove('DAY')

In [16]:
stock_list = ['GOOG', 'MSFT', 'AMZN', 'AAPL', 'SBUX']

In [67]:
bl_returns, Z = black_litterman_returns(stock_list, '2016-01-01', '2024-02-18')

How many view are there?:  3


********
View 1


By how much will the view affect the stocks in percentage? 8.53
Please write the number of stocks that are affected:  1


***********


Please write the affected stock symbol:  GOOG
Please enter weight in percentage:  100


...

----------
View 3


By how much will the view affect the stocks in percentage? 9.44
Please write the number of stocks that are affected:  1


***********


Please write the affected stock symbol:  SBUX
Please enter weight in percentage:  100


----------


In [68]:
bl_returns

,Black_litterman return
GOOG,0.139730
MSFT,0.173760
AMZN,0.176174
AAPL,0.150617
SBUX,0.108350


In [69]:
Z

,Z score
GOOG,-0.238989
MSFT,1.606730
AMZN,0.582674
AAPL,0.319623
SBUX,0.262050


In [78]:
Z['weights'] = Z['Z score'] /  Z.sum().values[0]

In [79]:
Z

,Z score,weights
GOOG,-0.238989,-0.094384
MSFT,1.606730,0.634548
AMZN,0.582674,0.230116
AAPL,0.319623,0.126229
SBUX,0.262050,0.103492


References:

SNP_tickers data : https://github.com/datasets/s-and-p-500-companies/blob/main/data/constituents.csv